In [97]:
import pandas as pd
import numpy as np

import pickle

import requests

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 

In [98]:
url = "https://gist.githubusercontent.com/AlexandreGazagnes/9018022652ba0933dd39c9df8a600292/raw/0845ef4c2df4806bb05c8c7423dc75d93e37400f/titanic_train_raw_csv"

In [99]:
df = pd.read_csv (url)

In [100]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [101]:
cols = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
df = df.loc[:, cols]
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [103]:
X = df.drop(columns="Survived")
y = df.Survived

In [104]:
X_train, X_test, y_train, y_test  = train_test_split(X, y)

In [105]:
"""
numeric_features = ["age", "fare"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
"""

'\nnumeric_features = ["age", "fare"]\nnumeric_transformer = Pipeline(\n    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]\n)\n'

In [106]:

categorical_features = ["Embarked", "Sex"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder='passthrough'
)

array([[ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    , 34.0208],
       [ 0.    ,  0.    ,  1.    , ...,  4.    ,  1.    , 39.6875],
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    , 13.    ],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  9.4833],
       [ 1.    ,  0.    ,  0.    , ...,  0.    ,  1.    , 83.1583],
       [ 0.    ,  0.    ,  1.    , ...,  1.    ,  2.    , 27.75  ]])

array([[ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    , 34.0208],
       [ 0.    ,  0.    ,  1.    , ...,  4.    ,  1.    , 39.6875],
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    , 13.    ],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  9.4833],
       [ 1.    ,  0.    ,  0.    , ...,  0.    ,  1.    , 83.1583],
       [ 0.    ,  0.    ,  1.    , ...,  1.    ,  2.    , 27.75  ]])

In [107]:
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("imputer", KNNImputer() ),
    ("scaler", StandardScaler()),
    ("estimator", RandomForestClassifier()),
    ])

In [108]:
grid = GridSearchCV(pipe, {}, cv=10, return_train_score=True, n_jobs=-1)

In [111]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Embarked',
                                                                          'Sex'])])),
                                       ('imputer', KNNImputer()),
                                       ('scaler', StandardScaler()),
                                       ('estimator',
                                        RandomForestClassifier())]),
             n_jobs=-1, param_grid={}, return_train_score=True)

In [114]:
res = pd.DataFrame(grid.cv_results_)
res = res.loc[:, [i for i in res.columns if "split" not in i ]]
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.263761,0.066473,0.025737,0.010075,{},0.794889,0.068172,1,0.985362,0.002328


In [115]:
grid.score(X_test, y_test)

0.820627802690583

## Save

In [124]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(grid)
    f.write(pk)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Embarked',
                                                                          'Sex'])])),
                                       ('imputer', KNNImputer()),
                                       ('scaler', StandardScaler()),
                                       ('estimator',
                                        RandomForestClassifier())]),
             n_jobs=-1, param_grid={}, return_train_score=True)